# Tavily Extract

[Tavily](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed. Tavily offers an [Extract](https://docs.tavily.com/api-reference/endpoint/extract) endpoint that can be used to extract content from a URLs.

## Overview

### Integration details
| Class | Package | Serializable | [JS support](https://js.langchain.com/docs/integrations/tools/tavily_search) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| TavilyExtract | [langchain-community](https://python.langchain.com/api_reference/community/index.html) | ❌ | ❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Native async | Return data | Pricing |
| :---: | :---: | :---: | :---: |
| ❌ | ✅ | content | 1,000 free searches / month | 


## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package.

In [1]:
%pip install -qU "langchain-community>=0.3.18" tavily-python

### Credentials

We also need to set our Tavily API key. You can get an API key by visiting [this site](https://app.tavily.com/sign-in) and creating an account.

In [2]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [3]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Instantiation

Here we show how to instantiate an instance of the Tavily extract tool. The tool accepts various parameters, including:

In [4]:
from langchain_community.tools import TavilyExtract

tool = TavilyExtract(
    extract_depth="basic",
    include_images=True,
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

The `TavilyExtract` tool accepts the following arguments:

- `urls` (required): A list of URLs to extract content from.
- `extract_depth` (optional): The depth of the extraction.
- `include_images` (optional): Whether to include images in the extraction.

NOTE: The optional arguments are available for ReAct agents to dynamically set, if you set a argument during instantiation and then Invoke the tool with a different value, the tool will use the value you passed during invokation. To read more about ReAct agents, check out the [ReAct agent](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/) documentation.


In [5]:
tool_msg = tool.invoke({"urls": ["https://en.wikipedia.org/wiki/Lionel_Messi"]})

# abbreviate for demo
tool_msg["results"][0]["raw_content"] = tool_msg["results"][0]["raw_content"][:400]
tool_msg

{'results': [{'url': 'https://en.wikipedia.org/wiki/Lionel_Messi',
   'raw_content': 'Lionel Messi\n\n\n\nLionel Andrés "Leo" Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughou',
   'images': []}],
 'failed_results': [],
 'response_time': 0.79}

### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [7]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"urls": ["https://en.wikipedia.org/wiki/Lionel_Messi"]},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

{"results": [{"url": "https://en.wikipedia.org/wiki/Lionel_Messi", "raw_content": "Contents\n\nLionel Messi\n\n\n\nMessi withArgentinaat the2022 FIFA World Cup\nPersonal information\nFull name | Lionel Andrés Messi[1]\nDate of birth | (1987-06-24)24 June 1987(age 37)[1]\nPlace of birth | Rosario, Argentina\nHeight | 1.70 m (5 ft 7 in)[1]\nPosition(s) | Forward\nTeam information\nCurrent team | Int


## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />


In [8]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

In [9]:
import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate(
    [
        ("system", f"You are a helpful assistant. The date today is {today}."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool])

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke(
    "extract information from the following url: https://en.wikipedia.org/wiki/Lionel_Messi"
)

AIMessage(content='It seems there was an issue extracting information from the Wikipedia page for Lionel Messi. You can try visiting the page directly [here](https://en.wikipedia.org/wiki/Lionel_Messi) for detailed information. If you have specific questions about Lionel Messi, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 339, 'total_tokens': 397, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'stop', 'logprobs': None}, id='run-30f558bb-83af-4b34-8540-9096c1cc8b58-0', usage_metadata={'input_tokens': 339, 'output_tokens': 58, 'total_tokens': 397, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## API reference

For detailed documentation of all TavilySearchResults features and configurations head to the API reference: